<h1>IMFDB Connector</h1>
<h3>Python script for populating a PostgreSQL database with article data from the Internet Movies and Firearms Database Wiki</h3>

In [33]:
# Imports
import psycopg2
import requests
import os
import re
from bs4 import BeautifulSoup
import pandas as pd #1.20 or above required

In [34]:
# Establish a connection to the database
cnx = psycopg2.connect(
    host="localhost",
    user="imfdb",
    password=os.environ.get("PG_IMFDB_PASSWORD"),
    database="imfdb"
)

# Create a cursor object
cursor = cnx.cursor()

<h3> MediaWiki API related functions</h3>
These are used to query the MW API for article data and populating the Postgres DB with the minimum necessary data for further processing.

In [35]:
def api_request(url):
    # Makes a get request to the specified API endpoint. A JSON response is expected.

    # Make a GET request to the IMFDB API endpoint
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the JSON data from the response
        return response.json()
    else:
        # Handle the error
        print(f"ERROR: api_request(): Request failed with status code: {response.status_code}")
        return None

In [36]:
def parse_page_by_id(pageid, prop, format):
    # Example: parse_page_by_id("215875","text", "json") to parse the wiki text of Weird Al Yankovic as json

    data = api_request(f"https://www.imfdb.org/api.php?action=parse&pageid={pageid}&prop={prop}&format={format}")

    # Error Handling
    if data is None:
        print("ERROR: parse_page_by_id(): Data is None!")
        return
    
    return data

In [37]:
def get_page_text_by_id(pageid):
    response = requests.get(f"https://www.imfdb.org/index.php?curid={pageid}")
    return str(response.text)
# We don't use the API anymore for this due to issues with the HTML it responds with
#def get_page_text_by_id(pageid):
#    data = parse_page_by_id(pageid, "text", "json")
#    return str(data["parse"]["text"]["*"])

In [38]:
def query_categorymembers(cmtitle, format):
    # Example: query_categorymembers("Category:Actor", "json") to query all actor pages as json.

    # Make a GET request to the IMFDB API endpoint
    data = api_request(f"https://www.imfdb.org/api.php?action=query&list=categorymembers&cmtitle={cmtitle}&format={format}")

    # Error Handling
    if data is None:
        print("ERROR: query_categorymembers(): Data is None!")
        return None

    #Initialize list
    categorymembers = []

    # Loop through the first batch of category members
    for member in data["query"]["categorymembers"]:
        print(f"Adding {member['title']}")
        categorymembers.append(member)

    # Continue fetching while there is something to be fetched
    while "continue" in data:
        data = api_request(f"https://www.imfdb.org/api.php?action=query&list=categorymembers&cmtitle={cmtitle}&format={format}&cmcontinue={data['continue']['cmcontinue']}")
        
        # Error Handling
        if data is None:
            print(f"ERROR: query_categorymembers(): Data is None in continuation batch {data['continue']['cmcontinue']}")
            return None
            
        # Loop through continuation batch:
        for member in data["query"]["categorymembers"]:
            print(f"Adding {member['title']}")
            categorymembers.append(member)

    return categorymembers

In [39]:
def populate_actors_table():
    actors = query_categorymembers("Category:Actor", "json")

    for actor in actors:

        actorpageid = str(actor['pageid'])
        actorurl = f"https://www.imfdb.org/index.php?curid={actorpageid}"
        actorpagecontent = get_page_text_by_id(actorpageid)
        actorname = str(actor['title'])
        if "Category:" in actorname:
            continue
        print(f"INSERTing: {actorname}, {actorpageid}")
        statement = "INSERT INTO actors (actorurl, actorpageid, actorpagecontent, actorname) VALUES (%s, %s, %s, %s)"
        cursor.execute(statement, (actorurl, actorpageid, actorpagecontent, actorname))
    
    cnx.commit()

In [40]:
def populate_movies_table():
    movies = query_categorymembers("Category:Movie", "json")

    for movie in movies:

        moviepageid = str(movie['pageid'])
        movieurl = f"https://www.imfdb.org/index.php?curid={moviepageid}"
        moviepagecontent = get_page_text_by_id(moviepageid)
        movietitle = str(movie['title'])
        if "Category:" in movietitle:
            continue
        print(f"INSERTing: {movietitle}, {moviepageid}")
        statement = "INSERT INTO movies (movieurl, moviepageid, moviepagecontent, movietitle) VALUES (%s, %s, %s, %s)"
        cursor.execute(statement, (movieurl, moviepageid, moviepagecontent, movietitle))
    
    cnx.commit()

In [41]:
def populate_tvseries_table():
    tvseries = query_categorymembers("Category:Television", "json")

    for series in tvseries:

        tvseriespageid = str(series['pageid'])
        tvseriesurl = f"https://www.imfdb.org/index.php?curid={tvseriespageid}"
        tvseriespagecontent = get_page_text_by_id(tvseriespageid)
        tvseriestitle = str(series['title'])
        if "Category:" in tvseriestitle:
            continue
        print(f"INSERTing: {tvseriestitle}, {tvseriespageid}")
        statement = "INSERT INTO tvseries (tvseriesurl, tvseriespageid, tvseriespagecontent, tvseriestitle) VALUES (%s, %s, %s, %s)"
        cursor.execute(statement, (tvseriesurl, tvseriespageid, tvseriespagecontent, tvseriestitle))
    
    cnx.commit()

In [42]:
def populate_firearms_table_minimally():
    firearms = query_categorymembers("Category:Gun", "json")

    for firearm in firearms:

        firearmpageid = str(firearm['pageid'])
        firearmurl = f"https://www.imfdb.org/index.php?curid={firearmpageid}"
        firearmpagecontent = get_page_text_by_id(firearmpageid)
        firearmtitle = str(firearm['title'])
        if "Category:" in firearmtitle:
            continue
        print(f"INSERTing: {firearmtitle}, {firearmpageid}")
        statement = "INSERT INTO firearms (firearmurl, firearmpageid, firearmpagecontent, firearmtitle) VALUES (%s, %s, %s, %s)"
        cursor.execute(statement, (firearmurl, firearmpageid, firearmpagecontent, firearmtitle))
    
    cnx.commit()

<h3>Database related functions</h3>
Once we have downloaded the necessary article data, we extract useful information from the HTML to fill the rest of our column.

In [43]:
# Some useful dictionaries #

firearms_dict = {
    "firearmid" : 0,
    "firearmurl" : 1,
    "parentfirearmid" : 2,
    "firearmpageid" : 3,
    "firearmpagecontent" : 4,
    "specificationid" : 5,
    "firearmtitle" : 6,
    "firearmversion" : 7,
    "isfamily" : 8,
    "isfictional" : 9
}

In [44]:
def get_page_content_from_db(pageid, table):
    # This only works with firearms that are NOT children
    if table in ["actors", "movies", "tvseries", "firearms"]:
        singular = table.rstrip("s")
        content = "{}pagecontent".format(singular)
        id = "{}pageid".format(singular)
    else:
        print("ERROR: get_page_content_from_db(): {} is not a valid table!".format(table))
        return None

    statement = "select {} from {} where {} = '{}';".format(content, table, id, pageid)
    if table == "firearms": # If the firearms table is queried, filter out child firearm rows
        statement = "select {} from firearms where {} = '{}' and parentfirearmid is null;".format(content, id, pageid)
    cursor.execute(statement)
    return cursor.fetchone()[0]

In [45]:
def get_page_content_from_db_by_uuid(uuid, table):
    # This works with all pages, including firearm children, but takes the uuid as key
    if table in ["actors", "movies", "tvseries", "firearms"]:
        singular = table.rstrip("s")
        content = "{}pagecontent".format(singular)
        id = "{}id".format(singular)
    else:
        print("ERROR: get_page_content_from_db(): {} is not a valid table!".format(table))
        return None

    statement = "select {} from {} where {} = '{}';".format(content, table, id, uuid)
    cursor.execute(statement)
    return cursor.fetchone()[0]

In [46]:
def update_firearms_isfictional():
    statement = "UPDATE firearms SET isfictional = FALSE WHERE NOT firearmtitle LIKE '(%) -%';"
    cursor.execute(statement)
    statement = "UPDATE firearms SET isfictional = TRUE WHERE firearmtitle LIKE '(%) -%';"
    cursor.execute(statement)
    cnx.commit()

In [47]:
def is_multi_gun_page(pageid):
    # Exceptions
    if (pageid == "464719" or pageid == "314208"): #Both of these have a table of contents despite being singles
        return False

    # If there are multiple h1s in an article, which are not See Also or Specification, it's a multi-gun page
    soup = soup = BeautifulSoup(get_page_content_from_db(pageid, "firearms"), 'html.parser')

    toctitle = soup.find("div", class_="toctitle") #If there is no table of contents, we don't need to check further, it's not multi-gun
    if toctitle is None:
        return False

    see_also = soup.find(id = "See_Also")
    if see_also is not None:
        if see_also.parent.name == "h1":
            see_also.parent.extract()

    spec = soup.find(id = "Specifications")
    if spec is not None:
        if spec.parent.name == "h1":
            spec.parent.extract()

    h1_tags = soup.find_all("h1")
    count = len(h1_tags)

    if count > 1:
        return True
    else:
        return False

In [48]:
def update_firearms_isfamily():
    # We assume a firearm is a family when it is named 'series' or has a multi-gun page
    keyword1 = "series"
    keyword2 = "Series"
    statement = "UPDATE firearms set isfamily = FALSE WHERE NOT (firearmtitle LIKE '%%%s%%' OR firearmtitle LIKE '%%%s%%' OR firearmtitle = 'Air Guns')" % (keyword1, keyword2)
    cursor.execute(statement)
    statement = "UPDATE firearms set isfamily = TRUE WHERE (firearmtitle LIKE '%%%s%%' OR firearmtitle LIKE '%%%s%%' OR firearmtitle = 'Air Guns')" % (keyword1, keyword2)
    cursor.execute(statement)

    statement = "SELECT * FROM firearms"
    cursor.execute(statement)
    firearms = cursor.fetchall()
    for firearm in firearms:
        if (is_multi_gun_page(pageid=firearm[3]) and firearm[8] == False): # If we have determined the article is multi-gun, it's a family
            statement = "UPDATE firearms set isfamily = TRUE WHERE firearmid = '%s'" % (firearm[0])
            cursor.execute(statement)
        if firearm[2] is not None: # Child firearms are never families 
            statement = "UPDATE firearms set isfamily = FALSE WHERE firearmid = '%s'" % (firearm[0])
            cursor.execute(statement)

    cnx.commit()

In [49]:
def get_number_of_specifications(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    spec_tags = soup.find_all(id=lambda value: value and value.startswith("Specifications"))
    spec_count = len(spec_tags)

    print("The number of h2 tags with 'Specifications' is:", spec_count)
    return spec_count

In [50]:
def strip_spec_list_item(item):
    index = item.index(":")
    return item[index + 1:].strip()

In [51]:
def get_single_specification(html_content, pageid=None):
    # Finds the first specification within any given HTML code

    spec_dict = {
        "production":None,
        "type":None,
        "caliber":None,
        "capacity":None,
        "fire_modes":None
    }

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all h1 headers in the content
    headers = soup.find_all("h1")

    if (headers is None or len(headers) == 0):
        if pageid is not None:
            print(f"ERROR: get_first_specification(): {pageid} does not contain any headers!")
            return

    # Page Title
    title = headers[0].text

    # Find the first Element with the Specifications id
    span = soup.find(id = "Specifications")
    if span is None:
        print("ERROR: get_first_specification(): No <span> tag with id = 'Specifications' was found!")
        return

    spec_lists = span.parent.find_next_siblings("ul")

    # Each spec row is its own unordered list with only a single list item
    if spec_lists is None:
        print("ERROR: get_first_specification(): No <ul> tags were found!")
        return

    specifications = [li.text for ul in spec_lists for li in ul.find_all('li')]

    if specifications is None:
        print("ERROR: get_first_specification(): No <li> tags were found!")
        return

    # Iterate through the list items and set a dict for each
    for item in specifications:
        if "Type:" in item:
            spec_dict["type"] = strip_spec_list_item(item)
        elif "Caliber:" in item:
            spec_dict["caliber"] = strip_spec_list_item(item)
        elif "Capacity:" in item:
            spec_dict["capacity"] = strip_spec_list_item(item)
        elif "Fire Modes:" in item:
            spec_dict["fire_modes"] = strip_spec_list_item(item)
    
    # Initialize with None so we can check later if a value was extracted from the HTML
    production = None

    # Determine whether a <p> tag containing a date exists
    possible_p_tag = span.parent.find_next_sibling()
    if possible_p_tag is not None:
        if possible_p_tag.name == "p" and re.match(r"\(\.*\d{4}.*\)",str(possible_p_tag.text)):
            production = possible_p_tag.text
            match = re.search(r"\(.*(\d{4})s?\s*(-\s|–\s)(\d{4}|Present)s?.*\)", production)
            if match is not None:
                production_year = match.group(1).strip()
                production_end_year = match.group(3).strip()
                production = f"({production_year} - {production_end_year})"
            # Add it to the specification
            spec_dict["production"] = production.rstrip("\n")
        
    return spec_dict

In [52]:
def articles_has_h1_variants(soup):
# Some firearm pages have variants ("Military, Civilian, etc.") as their h1, instead of different models
    if soup.find(id = "Specifications") is not None:
        first_specification_header = soup.find(id = "Specifications").parent
        if first_specification_header.name == "h3":
            return True
        else:
            return False
    else: # If the table of contents has a depth of 3 or more, we also assume variants
        toctitle = soup.find("div", class_="toctitle")
        if toctitle is not None:
            toc = toctitle.find_next_sibling("ul")
            regex = re.compile(r'(\d\.){2,}\d')
            for li in toc.find_all('li'):
                if regex.search(li.text):
                    return True
            return False
    return None

In [53]:
def generate_firearms_from_multi(html_content, url, pageid, parentuuid):
    # This function splits multi-gun pages at every h1, if it doesn't use h1s as variants, and at every h2, if it does.

    soup = BeautifulSoup(html_content, 'html.parser')

    firearmtitle = None
    content = ""
    version = None

    see_also = soup.find(id = "See_Also")
    if see_also is not None:
        if see_also.parent.name == "h1":
            see_also.parent.extract()

    spec = soup.find(id = "Specifications")
    if spec is not None:
        if spec.parent.name == "h1":
            h1spec= spec.parent.extract()

    if articles_has_h1_variants(soup): # If it has variants in h1...
        headers = soup.find_all("h1")
        headers.pop(0) #Skip the first one, since its the page heading
        for h1 in headers:
            version = h1.text
            headers2 = h1.find_next_siblings("h2")
            for h2 in headers2:
                slices = []
                slices.append(h2)
                firearmtitle = h2.text
                content = ""
                for slice in h2.find_next_siblings():
                    if (slice.name == "h2" or slice.name == "h1"):
                        break
                    slices.append(slice.extract())
                # Now that we have built up our slices of html, it's time to insert
                for slice in slices:
                    content = content+str(slice)
                if (content == "" or firearmtitle is None or version is None):
                    print(f"ERROR: generate_firearm_from_multi(): {pageid} produced empty version, content or title!")
                    continue
                if firearmtitle in ["Video Games", "Film", "Television", "Anime"]:
                    print(f"ERROR: generate_firearm_from_multi(): Version check failed for {pageid}!")
                    continue
                print(f"INSERTing: {firearmtitle}, {pageid}, {parentuuid}")
                statement = "INSERT INTO firearms (firearmurl, parentfirearmid, firearmpageid, firearmpagecontent, firearmtitle, isfamily, firearmversion) VALUES (%s, %s, %s, %s, %s, %s, %s)"
                cursor.execute(statement, (url, parentuuid, pageid, content,  firearmtitle, 'FALSE', version))
                
    else: # If it doesn't have variants in h1...
        headers = soup.find_all("h1")
        headers.pop(0) # Skip the first one
        for h1 in headers:
            slices = []
            slices.append(h1)
            firearmtitle = h1.text
            content = ""
            for slice in h1.find_next_siblings():
                if slice.name == "h1":
                    break
                slices.append(slice.extract())
            # Now that we have built up our slices of html, it's time to insert
            for slice in slices:
                content = content+str(slice)
            if (content == "" or firearmtitle is None):
                print(f"ERROR: generate_firearm_from_multi(): {pageid} produced empty content or title!")
                continue
            if firearmtitle in ["Video Games", "Film", "Television", "Anime"]:
                print(f"ERROR: generate_firearm_from_multi(): Version check failed for {pageid}!")
                continue
            print(f"INSERTing: {firearmtitle}, {pageid}, {parentuuid}")
            statement = "INSERT INTO firearms (firearmurl, parentfirearmid, firearmpageid, firearmpagecontent, firearmtitle, isfamily) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(statement, (url, parentuuid, pageid, content,  firearmtitle, 'FALSE'))

    if articles_has_h1_variants(soup) is None:
        print(f"generate_firearm_from_multi(): Version check failed for {pageid}!")

In [54]:
def generate_firearms_from_multis():
    statement = "SELECT * FROM firearms WHERE isfamily = 'True' AND parentfirearmid IS NULL;"
    cursor.execute(statement)
    firearms = cursor.fetchall()

    for firearm in firearms:
        generate_firearms_from_multi(html_content=firearm[4], url=firearm[1], pageid=firearm[3], parentuuid=firearm[0])
    cnx.commit()

In [55]:
def check_for_family_candidates():
# Debugging function
    statement = "SELECT * FROM firearms"
    cursor.execute(statement)
    firearms = cursor.fetchall()
    
    with open('candidates.txt', 'w') as writer:
        for firearm in firearms:
            if (is_multi_gun_page(pageid=firearm[3]) and firearm[8] == False):
                writer.write(f"{firearm[3]}\n")

In [56]:
def populate_specs_for_singles():
    statement = "SELECT * FROM firearms WHERE isfamily = 'False';"
    cursor.execute(statement)
    firearms = cursor.fetchall()

    for firearm in firearms:
        print(f"Fetching spec for: {firearm[3]}")
        spec = get_single_specification(html_content=firearm[4], pageid=firearm[3])
        if spec is not None:
            print(f"INSERTing: {firearm[3]} specification")
            statement = "INSERT INTO specifications (firearmid, type, caliber, capacity, firemode, productiontimeframe) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(statement, (firearm[0], spec["type"], spec["caliber"], spec["capacity"], spec["fire_modes"], spec["production"]))

In [57]:
def populate_specs_for_multies():
    # We handle family rows first, trying to determine whether there is a single spec in an h1 tag for the entire page
    statement = "SELECT * FROM firearms WHERE isfamily = 'True' and parentfirearmid IS NULL"
    cursor.execute(statement)
    firearms = cursor.fetchall()

    for firearm in firearms:
        html_content = firearm[4]
        soup = BeautifulSoup(html_content, "html.parser")
        spec_tag = soup.find_next(id = "Specifications") # Find the first spec
        if spec_tag is not None:
            if spec_tag.parent.name == "h1": # If it's nested in an h1...
                print(f"Fetching spec for: {firearm[3]}")
                spec = get_single_specification(html_content=firearm[4], pageid=firearm[3])
                if spec is not None:
                    print(f"INSERTing: {firearm[3]} specification")
                    statement = "INSERT INTO specifications (firearmid, type, caliber, capacity, firemode, productiontimeframe) VALUES (%s, %s, %s, %s, %s, %s)"
                    cursor.execute(statement, (firearm[0], spec["type"], spec["caliber"], spec["capacity"], spec["fire_modes"], spec["production"]))
    # Same procedure for the child rows
    statement = "SELECT * FROM firearms WHERE parentfirearmid IS NOT NULL"
    cursor.execute(statement)
    firearms = cursor.fetchall()

    for firearm in firearms:
        print(f"Fetching spec for: {firearm[3]}")
        spec = get_single_specification(html_content=firearm[4], pageid=firearm[3])
        if spec is not None:
            print(f"INSERTing: {firearm[3]} specification")
            statement = "INSERT INTO specifications (firearmid, type, caliber, capacity, firemode, productiontimeframe) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(statement, (firearm[0], spec["type"], spec["caliber"], spec["capacity"], spec["fire_modes"], spec["production"]))

In [58]:
def populate_specifications_table():
    populate_specs_for_singles()
    populate_specs_for_multies()
    cnx.commit()

In [59]:
def extract_dataframe_from_html_table(html_content, table_name, uuid):
    # This is used to find and extract the first table (valid names are 'Film' and 'Television' on any given page) and return it as a data frame
    if table_name not in ["Film", "Television"]:
        print(f"ERROR: extract_dataframe_from_html(): table {table_name} not found in list of valid table names!")
        return
    soup = BeautifulSoup(html_content, "html.parser")
    regex = re.compile(fr'^{table_name}(_\d*)?')
    span = soup.find('span', {'id': regex})
    if span is None:
        print(f"WARNING: extract_dataframe_from_html_table(): No span with id {table_name} was found in the html content of {uuid}!")
        return None
    table = span.parent.find_next_sibling("table")
    if table is None:
        print(f"ERROR: extract_dataframe_from_html_table(): No table was found in the html content of {uuid}!")
        return None
    just_the_table = str(table)
    df = pd.read_html(just_the_table, extract_links='body')
    return df[0]

In [60]:
def skip_this_uuid_DEBUG(uuid):
    if uuid in ['dceac3cc-9e05-4f78-a2cb-6655387e764f']:
        return True
    else:
        return False

In [61]:
def populate_movies_actors_firearms_table():
    statement = "SELECT * FROM firearms"
    cursor.execute(statement)
    firearms = cursor.fetchall()

    for firearm in firearms:
        uuid = firearm[0]
        html = get_page_content_from_db_by_uuid(uuid, "firearms")
        df = extract_dataframe_from_html_table(html, "Film", uuid)
        if df is None:
            continue
        
        print(f"CURRENTLY WORKING ON: {uuid}")
        if skip_this_uuid_DEBUG(uuid):
            print("Skipping...")
            continue

        # Columns don't have consistent naming, so we have do go through this and match them with regex
        title_col_name = actor_col_name = character_col_name = note_col_name = date_col_name = None

        title_regex = re.compile('.*(Title|Film|Movie|Titla).*', re.IGNORECASE)
        actor_regex = re.compile('.*Actor.*', re.IGNORECASE)
        character_regex = re.compile('.*(Character|Charcter).*', re.IGNORECASE)
        note_regex = re.compile('.*(Note|Notation).*', re.IGNORECASE)
        date_regex = re.compile('.*(Date|Year).*', re.IGNORECASE)

        for col_name in df.columns:
            if title_regex.match(col_name):
                title_col_name = col_name
            elif actor_regex.match(col_name):
                actor_col_name = col_name
            elif character_regex.match(col_name):
                character_col_name = col_name
            elif note_regex.match(col_name):
                note_col_name = col_name
            elif date_regex.match(col_name):
                date_col_name = col_name
        
        if any(var is None for var in [title_col_name, actor_col_name, character_col_name, note_col_name, date_col_name]):
            print(f"WARNING: populate_movies_actors_firearms_table(): The html content in '{uuid}' has one or more unmatched columns in its 'Film' table")

        # Extract the row values and INSERT them
        for i in range(len(df.index)):
            title = actor = character = note = date = "NULL"
            title = df[title_col_name][i]
            if actor_col_name is not None:
                actor = df[actor_col_name][i]
            character = df[character_col_name][i]
            if note_col_name is not None:
                note = df[note_col_name][i]
            date = df[date_col_name][i]
            
            # Some of the values may be NaN. We set those to NULL
            if pd.isna(title):
                title = "NULL"
            if pd.isna(actor):
                actor = "NULL"
            if pd.isna(character):
                character = "NULL"
            if pd.isna(note):
                note = "NULL"
            if pd.isna(date):
                date = "NULL"

            print(f"====\n{title}\n{actor}\n{character}\n{note}\n{date}\n====")
    return

In [62]:
def populate_tvseries_actors_firearms_table():
    return

<h3>Corner and edge cases:<h3>

In [63]:
# Corner-case handling:

# X HK416 second variant has full html for some reason ()
# X Author of Flammenwerfer 35 page is retarded and can't spell table headers right
# X Same for DefTech 37mm GL
# X Film and Tele tables have Notes only as an option
# X Film and tele tables may sometimes be in Spanish
# X The notes column may contain the string "Note" || "Notes" || "Notation" || Notations
# X actors my be NaN in df
# X https://www.imfdb.org/index.php?curid=62 has just a single Spec at the beginning of the page
# Child guns are currently NULL in isfictional column. -> isfictional should default to false
# X https://www.imfdb.org/index.php?curid=464719 Sage BML-37 has a table of contents despite being a single
# https://www.imfdb.org/index.php?curid=348107 HK AG Grenade Launchers are a weird mix of versioned and non-versioned, ie live-fire models are non-versioned, non-firing replicas are versioned
# X https://www.imfdb.org/index.php?curid=3564 SIG P210 has its Video Game table nested in the Television segment
# X https://www.imfdb.org/index.php?curid=314208 Flintlock Musket has a table of contents despite being a single


<h3>Main<h3>

In [64]:
### We do stuff here: ###

#Populate the database skeleton:

#populate_actors_table()
#populate_movies_table()
#populate_tvseries_table()
#populate_firearms_table_minimally()

#Finalize the firearms table:

#update_firearms_isfictional()
#update_firearms_isfamily()
#generate_firearms_from_multis()

# Populate the specifications table:

#populate_specifications_table()

# Populate junction tables:
#populate_movies_actors_firearms_table()

#DEBUG

html = get_page_content_from_db_by_uuid("b7704d2f-1279-47ab-a897-be9c22e8cd8c", "firearms")
df = extract_dataframe_from_html_table(html, "Film", "b7704d2f-1279-47ab-a897-be9c22e8cd8c")
df
#title_regex = re.compile('.*Title.*', re.IGNORECASE)
#for col_name in df.columns:
#    if title_regex.match(col_name):
#        title_col_name = col_name
#df[title_col_name][0]

,"(Show Title, None)","(Actor, None)","(Character, None)","(Note, None)","(Air Date, None)"
0,"(Deadlier Than the Male, /wiki/Deadlier_Than_t...","(Elke Sommer, /wiki/Elke_Sommer)","(Irma Eckman, None)","(, None)","(1967, None)"


In [65]:
# Close database connection
cursor.close()
cnx.close()